**The purpose of this notebook will be to build the Multinomial Naive Bayes algorithm from scratch. Then, it will be compared to sklearn's equivalent**

In [1]:
import numpy as np

In [2]:
from sklearn.datasets import fetch_20newsgroups 

In [3]:
newsgroups_train = fetch_20newsgroups(subset='train')

In [4]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

For sake of somplicity, let's narrow down our features into 3 categories. How about space, hockey, guns...3 completely different classes

In [5]:
categories=['rec.sport.hockey','sci.space', 'talk.politics.guns' ]

In [6]:
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)

In [7]:
newsgroups_train.data[5]

"From: e8l6@jupiter.sun.csd.unb.ca (Rocket)\nSubject: Dear Montana@pinetree.org  Re: Hockey Pool\nDistribution: rec.sport.hockey\nOrganization: University of New Brunswick\nLines: 15\n\n\n\n   Hi there, I can't seem to get mail to you. Can you tell me your entire\nadress, or even your dotted decimal address?\n\n   (ie. 131.202.3.10)\n\n   Thanks,\n           rocket@calvin.cs.unb.ca \n\n-- \n\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-\n-                                                                           -\n-    Maurice Richard                                                        -\n"

Since these seem to be emails. We don't need the headers, footers, or quotes in the text. These will only serve to overfit our model and not allow it to be best at guessing new material!

In [8]:
newsgroups_train = fetch_20newsgroups(subset='train',remove=('headers', 'footers', 'quotes'),categories=categories)

In [10]:
newsgroups_train.data[5]

"Hi there, I can't seem to get mail to you. Can you tell me your entire\nadress, or even your dotted decimal address?\n\n   (ie. 131.202.3.10)\n\n   Thanks,\n           rocket@calvin.cs.unb.ca \n\n-- "

Ok now we can start building our word count/document matrix 

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
vectorizer=CountVectorizer(stop_words='english') #remove stop words 

In [13]:
vectors=vectorizer.fit_transform(newsgroups_train.data)

In [14]:
vectors.shape #so there are ~28,000 unique words for all 1739 documents

(1739, 27558)

In [15]:
import pandas as pd

In [23]:
df=pd.DataFrame(vectors.todense())

In [24]:
df.columns=vectorizer.get_feature_names()

In [25]:
df['target_label']=pd.Series(newsgroups_train.target) #add target labels 

In [27]:
df.head()

,00,000,0000,00000,000000,000062david42,000152,00041032,0004136,0004246,...,zx,zx6wre,zxp,zxqi,zy,zyg,zz,zz_g9q3,zzzzzz,target_label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Ok, now we have a DataFrame containing all the unique words and counts for each of our documents. Let's do a test/train split then we can build the model

In [28]:
from sklearn.model_selection import train_test_split

In [125]:
x=df.drop('target_label', axis=1)
y=df['target_label']

In [31]:
x_train, x_test, y_train, y_test=train_test_split(x,y, random_state=123)

In [126]:
count_sample = x.shape[0]
        
x=np.array(x)

separated = [[x for x, t in zip(x, y) if t == c] for c in np.unique(y)]
#separates the documents by class

class_log_prior = [np.log(len(i) / count_sample) for i in separated]
#need to use log prior probability just in case value is too small (avoids underflow)

count = np.array([np.array(i).sum(axis=0) for i in separated]) + 1

feature_log_prob = np.log(count / count.sum(axis=1)[np.newaxis].T)

In [133]:
feature_log_prob

array([[ -6.77020036,  -7.983223  , -11.47973056, ..., -11.47973056,
        -11.47973056, -10.78658338],
       [ -7.96716463,  -6.9310727 , -10.3650599 , ..., -11.46367219,
        -11.46367219, -11.46367219],
       [ -8.18451375,  -6.52058766, -11.40338958, ..., -10.30477729,
        -10.7102424 , -11.40338958]])

In [149]:
probs=[(feature_log_prob * x).sum(axis=1) + class_log_prior
            for x in x]

In [150]:
probs

[array([-412.37055277, -376.93754977, -401.25555884]),
 array([-158.51042572, -188.01519983, -185.40318022]),
 array([-150.55706808, -139.36991704, -147.95407871]),
 array([-2622.44784922, -2498.50272339, -2218.59746934]),
 array([-783.13723503, -697.58254034, -789.24700977]),
 array([-156.11323179, -154.95671289, -162.7271153 ]),
 array([-715.15684111, -669.08516019, -613.65286896]),
 array([-7816.94638913, -6632.66241591, -7442.95689533]),
 array([-355.22443545, -321.89145951, -344.07018487]),
 array([-134.60738884, -137.9218046 , -136.03352234]),
 array([-263.79678517, -258.39560948, -229.7080635 ]),
 array([-669.67049668, -643.52200374, -561.74537808]),
 array([-1.06413586, -1.07587112, -1.15844654]),
 array([-1067.40048085,  -920.46314281, -1020.30117938]),
 array([-29.2117584 , -28.80759378, -27.71892265]),
 array([-165.91260008, -163.1362082 , -155.03330544]),
 array([-92.97689536, -87.32193594, -88.88978704]),
 array([-912.01699938, -871.1322117 , -790.26432546]),
 array([-206.

In [154]:
np.argmax(probs, axis=1).shape

(1739,)

Now let's build the Multinomial Naive Bayes Class

In [209]:
class Multinomial_Naive_Bayes():
    
    def __init__(self, alpha=1.0):
        self.alpha=alpha #this is smoothing parameter to make sure a  feature probability of 0 
                         #doesn't cause entire probability to be 0
            
    def fit(self, x,y):
        
        count_sample = x.shape[0]
        
        x=np.array(x)
        
        separated = [[x for x, t in zip(x, y) if t == c] for c in np.unique(y)]
        #separates the documents by class
                
        self.class_log_prior = [np.log(len(i) / count_sample) for i in separated]
        #need to use log prior probability just in case value is too small (avoids underflow)
        
        count = np.array([np.array(i).sum(axis=0) for i in separated]) + self.alpha
        #get word counts for each document 
        
        self.feature_log_prob = np.log(count / count.sum(axis=1)[np.newaxis].T)
        #get log probabilities for each feature (word) for each class

        
    def predict(self, x):
       
        x = np.asarray(x, dtype='float64')
        
        probs= [(self.feature_log_prob * x).sum(axis=1) + self.class_log_prior
            for x in x]
        
        #get probabiltities for each document belonging to each class
        
        self.predicted_labels=np.argmax(probs, axis=1)
        
        #find the index position (label) with highest probability 
        
        return self.predicted_labels
    
    def score(self, x,y):
        
        y=np.asarray(y)
        
        predicted=self.predict(x)
        
        score=0
        
        for i in range(len(y)):
            if predicted[i] == y[i]: #check to see if predicted labels match
                score+=1
        
        return (score)/len(y) #return accuracy score
         
    
        
 
        
        
        
        
        

In [210]:
nb=Multinomial_Naive_Bayes()

In [211]:
nb.fit(x_train, y_train)

In [212]:
nb.predict(x_test)

array([2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 2, 1, 0, 1, 1, 0, 2, 0, 2, 2,
       2, 1, 1, 0, 2, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 1, 1, 2, 2, 0, 0, 2,
       1, 2, 1, 0, 2, 0, 2, 0, 1, 2, 0, 1, 2, 0, 0, 0, 0, 1, 1, 2, 2, 2,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 2, 0, 1, 2, 0, 1, 1, 0, 0, 2, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 2, 2, 0, 2, 0, 2, 2, 1, 0, 0, 1, 0, 0, 2, 0, 0,
       2, 1, 1, 0, 1, 1, 0, 1, 2, 2, 1, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 1, 2, 1, 1, 2, 0, 2, 2, 0, 2, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 2, 1, 0, 2, 1, 0, 1, 2, 0, 1, 1, 2, 2, 0, 0,
       0, 0, 2, 0, 1, 1, 2, 2, 1, 0, 1, 1, 0, 1, 1, 1, 2, 0, 2, 0, 1, 2,
       1, 2, 2, 0, 0, 0, 2, 0, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 0, 0,
       0, 0, 1, 2, 2, 0, 2, 2, 2, 1, 2, 0, 2, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 2, 2, 1, 1, 0, 1, 2, 2, 0, 0, 1, 0, 2, 1, 1, 2, 1,
       1, 0, 1, 0, 0, 0, 2, 2, 2, 1, 0, 2, 1, 0, 2, 0, 2, 2, 1, 1, 1, 1,
       1, 2, 2, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 2, 1,

In [213]:
nb.score(x_test, y_test)

0.9103448275862069

91% accuracy. Not too shabby, now let's see how sklearn does

In [214]:
from sklearn.naive_bayes import MultinomialNB

In [215]:
nb_sk=MultinomialNB()

In [216]:
nb_sk.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [217]:
nb_sk.score(x_test,y_test)

0.9103448275862069

Our custom algorithm achieves the same accuracy as sklearn. Hooray!!!